In [8]:
import openai
import base64
import json
from dotenv import load_dotenv
import os

load_dotenv()
key = os.getenv("OPEN_API_KEY")

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

image_path = "WhatsApp Image 2025-04-05 at 13.19.46_a48c2e13.jpeg"
base64_image = encode_image(image_path)
json_schema = {
    "type": "object",
    "properties": {
        "keywords": {
            "type": "array",
            "description": "List of keywords that describe the content of the image.",
            "items": {"type": "string"},
            "minItems": 5,
            "maxItems": 5
        }
    },
    "required": ["keywords"],
    "additionalProperties": False
}

prompt = (
    "I am sending an image of an incident. Find out whether the image indicates an emergency or not and "
    "provide exactly five keywords describing the situation. Choose exclusively from keywords such as "
    "'accident', 'not accident', 'emergency', 'fire', 'flood', etc. Return only a JSON object following "
    "the schema provided."
)

client = openai.OpenAI(api_key=key)

response = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt},
                {"type": "input_image", "image_url": f"data:image/jpeg;base64,{base64_image}"}
            ]
        }
    ],
    text={
        "format": {
            "name":"keywords",
            "type": "json_schema",
            "schema": {
                "type": "object",
                "strict": True,
                "name": "keywords",
                "properties": {
                    "keywords": {
                        "type": "array",
                        "items": {"type": "string"}
                    },
                    "decision":{
                        "type": "string",
                        "enum": ["emergency", "not emergency"],
                        "description": "Decision on whether the image indicates an emergency or not."
                    }
                },
                "required": ["keywords","decision"],
                "additionalProperties": False
            }
        }
    }
)

print(response)



Response(id='resp_67f188f703b48192b94b6990faccc75f0a866589bbd9fb30', created_at=1743882487.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[ResponseOutputMessage(id='msg_67f1890418708192b3812cd5416756110a866589bbd9fb30', content=[ResponseOutputText(annotations=[], text='{"keywords":["fire","emergency","accident","evacuation","panic"],"decision":"emergency"}', type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, generate_summary=None), status='completed', text=ResponseTextConfig(format=ResponseFormatTextJSONSchemaConfig(name='keywords', schema_={'type': 'object', 'strict': True, 'name': 'keywords', 'properties': {'keywords': {'type': 'array', 'items': {'type': 'string'}}, 'decision': {'type': 'string', 'enum': ['emergency

In [10]:
json.loads(response.output_text) # Convert the response to JSON format

{'keywords': ['fire', 'emergency', 'accident', 'evacuation', 'panic'],
 'decision': 'emergency'}

In [11]:
from flask import Flask, request, jsonify
import openai
import base64
import json
from dotenv import load_dotenv
import os
import requests

load_dotenv()

app = Flask(__name__)

key = os.getenv("OPEN_API_KEY")
openai.api_key = key

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
@app.route('/analyze', methods=['POST'])
def analyze():
    if 'image' not in request.files:
        return jsonify({"error": "No image provided"}), 400

    image_file = request.files['image']
    base64_image = encode_image(image_file)

    prompt = (
        "I am sending an image of an incident. Find out whether the image indicates an emergency or not and "
        "provide exactly five keywords describing the situation. Choose exclusively from keywords such as "
        "'accident', 'not accident', 'emergency', 'fire', 'flood', etc. Return only a JSON object following "
        "the schema provided."
    )
    
    client = openai.OpenAI(api_key=key)
    
    response = client.responses.create(
        model="gpt-4o",
        input=[
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": prompt},
                    {"type": "input_image", "image_url": f"data:image/jpeg;base64,{base64_image}"}
                ]
            }
        ],
        text={
            "format": {
                "name":"keywords",
                "type": "json_schema",
                "schema": {
                    "type": "object",
                    "strict": True,
                    "name": "keywords",
                    "properties": {
                        "keywords": {
                            "type": "array",
                            "items": {"type": "string"}
                        },
                        "decision":{
                            "type": "string",
                            "enum": ["emergency", "not emergency"],
                            "description": "Decision on whether the image indicates an emergency or not."
                        }
                    },
                    "required": ["keywords","decision"],
                    "additionalProperties": False
                }
            }
        }
    )

    try:
        result = json.loads(response.output_text)
    except Exception as e:
        return jsonify({"error": "Failed to parse response", "details": str(e)}), 500
    
    if result.get("decision") == "emergency":
        payload = {
            "event": "emergency_detected",
            "keywords": result.get("keywords", [])
        }

        return jsonify(payload), 200
    else:
        return jsonify({"message": "No emergency detected", "keywords": result.get("keywords", [])}), 200



